In [1]:
import numpy as np
import scipy.optimize as opt
from typing import List, Dict, Union

from functools import partial

In [10]:
senior_account_people = 2
senior_account_hour_per_year = 720
senior_cost_per_hour = 20

junior_account_people = 2
junior_account_hour_per_year = 1720
junior_cost_per_hour = 7

creative_people = 4
creative_hour_per_year = 1720
creative_cost_per_hour = 12


total_hour_disposal = {"senior_account": senior_account_people * senior_account_hour_per_year,
                "junior_account": junior_account_people * junior_account_hour_per_year,
                "creative": creative_people * creative_hour_per_year}

costs = {"senior_account": senior_cost_per_hour,
        "junior_account": junior_cost_per_hour,
        "creative": creative_cost_per_hour}


total_hour_disposal, costs

({'senior_account': 1440, 'junior_account': 3440, 'creative': 6880},
 {'senior_account': 20, 'junior_account': 7, 'creative': 12})

In [17]:
project_a_type = {"return": 1*1e4,
                 "creative_hour": 10,
                 "senior_hour": 20}

project_a_type["cost"] = project_a_type["creative_hour"] * creative_cost_per_hour + project_a_type["senior_hour"] * senior_cost_per_hour

project_b_type = {"return": 9*1e3,
                 "creative_hour": 20,
                 "junior_hour": 10}
project_b_type["cost"] = project_b_type["creative_hour"] * creative_cost_per_hour + project_b_type["junior_hour"] * junior_cost_per_hour

project_list = [project_a_type, project_b_type]
project_list

[{'return': 10000.0, 'creative_hour': 10, 'senior_hour': 20, 'cost': 520},
 {'return': 9000.0, 'creative_hour': 20, 'junior_hour': 10, 'cost': 310}]

In [18]:
def loss_function(t: np.ndarray,
                  projects: List[Dict[str, Union[float, int]]]):
    costs = 0.0
    returns = 0.0
    costs += t[0] * (projects[0]["cost"])
    returns += t[0] * projects[0]["return"]
    #project_B
    costs += t[1] * (projects[1]["cost"])
    returns += t[1] * projects[1]["return"]
    
    return returns - costs
    
    
def creative(t):
    return -(t[0] * project_a_type["creative_hour"] + t[1] * project_b_type["creative_hour"]) + total_hour_disposal["creative"]

def senior(t):
    return -t[0] * project_a_type["senior_hour"] + total_hour_disposal["senior_account"]

def junior(t):
    return -t[1] * project_b_type["junior_hour"] + total_hour_disposal["junior_account"]


def verify_hour(t):
    return (t[1] * project_b_type["junior_hour"] <= total_hour_disposal["junior_account"]) & (t[0] * project_a_type["senior_hour"] <=  total_hour_disposal["senior_account"]) & (t[0] * project_a_type["creative_hour"] + t[1] * project_b_type["creative_hour"] <= total_hour_disposal["creative"])

In [19]:
matrix = [-project_a_type["return"] + project_a_type["creative_hour"]*costs["creative"] + project_a_type["senior_hour"]*costs["senior_account"],
     -project_b_type["return"] + project_b_type["creative_hour"]*costs["creative"] + project_b_type["junior_hour"]*costs["junior_account"],
    ]
A = [[project_a_type["creative_hour"], project_b_type["creative_hour"]],
     [project_a_type["senior_hour"], 0],
     [0,project_b_type["junior_hour"]]]
b = [total_hour_disposal["creative"], total_hour_disposal["senior_account"], total_hour_disposal["junior_account"]]


# define the upper bound and the lower bound
R = (0, None)
T = (0, None)
# Implementing the Simplex Algorithm
# Solve the problem by Simplex method in Optimization
res = opt.linprog(matrix, A_ub=A, b_ub=b,  bounds=(R, T), method='highs', options={"disp": True})  # linear programming p[roblem
print(res) # print results

           con: array([], dtype=float64)
 crossover_nit: 0
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -3359080.0
       ineqlin:  marginals: array([-434.5 , -256.75,   -0.  ])
  residual: array([  0.,   0., 360.])
         lower:  marginals: array([0., 0.])
  residual: array([ 72., 308.])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 0
         slack: array([  0.,   0., 360.])
        status: 0
       success: True
         upper:  marginals: array([0., 0.])
  residual: array([inf, inf])
             x: array([ 72., 308.])


In [29]:
result = res.x
for i in range(len(result)):
    result[i] = int(result[i])
result[0] = 70
result[1] = 81
#result[1] = 68
print("Allocation", result)
print("P&L", loss_function(result, project_list)/1e5,"k")
print("Hour constraint verified", verify_hour(result))

Allocation [70. 81.]
P&L 155.7413 k
Hour constraint verified True


In [22]:
result[0]*(project_a_type["return"] - project_a_type["creative_hour"]*costs["creative"] + project_a_type["senior_hour"]*costs["senior_account"])

8634240.0

In [23]:
(result[1] * project_b_type["junior_hour"] <= total_hour_disposal["junior_account"]) 

True

In [24]:
(result[0] * project_a_type["senior_hour"] <=  total_hour_disposal["senior_account"]) 

True

In [25]:
result[0] * project_a_type["creative_hour"] + result[1] * project_b_type["creative_hour"]  

6880.0

In [26]:
total_hour_disposal["creative"]

6880

In [20]:
senior = 1720*2
senior


3440